In [2]:
import os


In [3]:
import torch.nn as nn
import torch.optim as optim
import torch as T
from torch.distributions.categorical import Categorical
import numpy as np



class AgentNetwork(nn.Module):
    def __init__(self,input_dims,action_dim,lr,layer1=256,layer2=256,weight_file='weightFiles/ppo_discrete'):
        super(AgentNetwork,self).__init__()
        self.checkpoint_file = os.path.join(weight_file,'ppo_actor_weight')
        #TOCHECK: *input_dims vs input_dims
        self.actor = nn.Sequential(
                nn.Linear(*input_dims,layer1),
                nn.ReLU(),
                nn.Linear(layer1,layer2),
                nn.ReLU(),
                nn.Linear(layer2,action_dim),
                nn.Softmax(dim=-1)               
        )
        
        self.optimizer = optim.Adam(self.parameters(),lr=lr)
        self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')
        self.to(self.device)
    
    def forward(self,state):
        dist = self.actor(state)
        #TOCHECK: what does categorical do
        dist = Categorical(dist)
        return dist
    
    def save_checkpoint(self):
        T.save(self.state_dict(),self.checkpoint_file)
    
    def load_checkpoint(self):
        self.load_state_dict(T.load(self.checkpoint_file))

class CriticNetwork(nn.Module):
    def __init__(self,input_dims,lr,layer1=256,layer2=256,weight_file='weightFiles/ppo_discrete'):
        super(CriticNetwork,self).__init__()
        self.checkpoint_file = os.path.join(weight_file,'ppo_critic_weight')
        self.critic = nn.Sequential(
                nn.Linear(*input_dims,layer1),
                nn.ReLU(),
                nn.Linear(layer1,layer2),
                nn.ReLU(),
                nn.Linear(layer2,1)
        )
        self.optimizer = optim.Adam(self.parameters(),lr=lr)
        self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')
        self.to(self.device)
        
    def forward(self,state):
        value = self.critic(state)
        return value
    
    def save_checkpoint(self):
        T.save(self.state_dict(),self.checkpoint_file)
    
    def load_checkpoint(self):
        self.load_state_dict(T.load(self.checkpoint_file))
        
    
    
    
        
        
        
        
        


In [4]:
class ReplayMemory:
    def __init__(self,batch_size):
        self.states = []
        self.probs=[]
        self.vals=[]
        self.actions=[]
        self.rewards=[]
        self.dones=[]
        self.batch_size = batch_size
    
    def store_memory(self,state,action,prob,val,reward,done):
        self.states.append(state)
        self.actions.append(action)
        self.probs.append(prob)
        self.vals.append(val)
        self.rewards.append(reward)
        self.dones.append(done)
        
    def clear_memory(self):
        self.states = []
        self.probs=[]
        self.vals=[]
        self.actions=[]
        self.rewards=[]
        self.dones=[]
        
    def generate_batches(self):
        n_states = len(self.states)
        batches=[]
        i=0
        indices = np.arange(n_states,dtype = np.int64)
        np.random.shuffle(indices)
        for i in range(n_states):
            batches.append(indices[i:i+self.batch_size])
            i+=self.batch_size
        return np.array(self.states),\
                np.array(self.actions),\
                np.array(self.probs), \
                np.array(self.vals),\
                np.array(self.rewards), \
                np.array(self.dones),\
                batches
    

In [7]:
class Agent:
    def __init__(self,n_actions,input_dims,gamma=0.99,lr=0.0003,lambda_factor=0.95,policy_clip=0.2,batch_size=64,n_epochs=10):
        self.gamma = gamma
        self.policy_clip = policy_clip
        self.n_epochs = n_epochs
        self.lambda_factor = lambda_factor
        self.actor = AgentNetwork(action_dim = n_actions,input_dims = input_dims,lr = lr)
        self.critic = CriticNetwork(input_dims,lr)
        self.memory = ReplayMemory(batch_size)
        
    def remember(self,state,action,prob,val,reward,done):
        self.memory.store_memory(state,action,prob,val,reward,done)
    
    def save_models(self):
        print("saving model file")
        self.actor.save_checkpoint()
        self.critic.save_checkpoint()
    
    def load_models(self):
        print("loading model file")
        self.actor.load_checkpoint()
        self.critic.load_checkpoint()
        
    def choose_action(self,observation):
        state = T.tensor([observation],dtype=T.float).to(self.actor.device)
        dist = self.actor(state)
        value = self.critic(state)
        #TOCHECK what is the datatype action 
        action = dist.sample()
        #TOCHECK what does squeeze do
        
        probs = T.squeeze(dist.log_prob(action)).item()

        
        action = T.squeeze(action).item()
        value  = T.squeeze(value).item()
        return action,probs,value
        
    def learn(self):
        for _ in range(self.n_epochs):
            state_arr,action_arr,probs_arr,vals_arr, \
            rewards_arr,dones_arr,batches = self.memory.generate_batches()
            
            advantages=np.zeros_like(rewards_arr)
            
            for t in reversed(range(len(state_arr)-1)):
                advantages[t] = rewards_arr[t]+self.gamma*vals_arr[t+1]*(1-int(dones_arr[t]))-vals_arr[t] + self.gamma*self.lambda_factor*advantages[t+1]
                
            advantages = T.tensor(advantages).to(self.actor.device)
            values = T.tensor(vals_arr).to(self.actor.device)
            
            for batch in batches:
                states = T.tensor(state_arr[batch],dtype = T.float).to(self.actor.device)
                actions = T.tensor(action_arr[batch],dtype = T.float).to(self.actor.device)
                old_probs = T.tensor(probs_arr[batch],dtype = T.float).to(self.actor.device)
                dist = self.actor(states)
                new_probs = dist.log_prob(actions)
                #TOCHECK: what do exp() do
                
                prob_ratio = new_probs.exp()/old_probs.exp()
                
                weighted_prob = advantages[batch]*(prob_ratio)
                
                weighted_clipped_probs = T.clamp(prob_ratio,1-self.policy_clip,1+self.policy_clip)*advantages[batch]
                
                actor_loss = - T.min(weighted_clipped_probs,weighted_prob).mean()
                
                critic_values = self.critic(states)
                #TOCHECK what does squeeze do here
                critic_values = T.squeeze(critic_values)
                
                desired_state_values = advantages[batch]+values[batch]
                critic_loss = (desired_state_values-critic_values)**2
                critic_loss = critic_loss.mean()
                
                total_loss = actor_loss+critic_loss*0.5
                
                self.actor.optimizer.zero_grad()
                self.critic.optimizer.zero_grad()
                total_loss.backward()
                self.actor.optimizer.step()
                self.critic.optimizer.step()
        self.memory.clear_memory() 
    

In [8]:
import gym


env = gym.make('CartPole-v1')
print( env.observation_space.shape)
agent = Agent(n_actions = env.action_space.n,input_dims = env.observation_space.shape,batch_size=5,n_epochs=4)
N = 20
episodes = 3000
n_step=0
learn_iters=0
best_score = 0
score_history=[]

for ep in range(episodes):
    state = env.reset()
    done = False
    score = 0
    while not done:
        action,prob,val = agent.choose_action(state)
        next_state,reward,done,_ = env.step(action)
        
        score+=reward
        n_step+=1
        agent.remember(state,action,prob,val,reward,done)
        state = next_state
        if n_step %N == 0:
            agent.learn()
            learn_iters+=1
    score_history.append(score)
    avg_score = np.mean(score_history[-100:])
    
    if avg_score > best_score:
        best_score = avg_score
        agent.save_models()
    print(f"episode {ep} current score {score} avg score {avg_score} best_score {best_score}")
    
        
        
        


(4,)
saving model file
episode 0 current score 11.0 avg score 11.0 best_score 11.0
saving model file
episode 1 current score 27.0 avg score 19.0 best_score 19.0
saving model file
episode 2 current score 36.0 avg score 24.666666666666668 best_score 24.666666666666668
episode 3 current score 23.0 avg score 24.25 best_score 24.666666666666668
episode 4 current score 20.0 avg score 23.4 best_score 24.666666666666668
episode 5 current score 30.0 avg score 24.5 best_score 24.666666666666668
episode 6 current score 13.0 avg score 22.857142857142858 best_score 24.666666666666668
episode 7 current score 29.0 avg score 23.625 best_score 24.666666666666668
saving model file
episode 8 current score 102.0 avg score 32.333333333333336 best_score 32.333333333333336
saving model file
episode 9 current score 33.0 avg score 32.4 best_score 32.4
episode 10 current score 27.0 avg score 31.90909090909091 best_score 32.4
saving model file
episode 11 current score 90.0 avg score 36.75 best_score 36.75
episod

episode 89 current score 28.0 avg score 99.18888888888888 best_score 131.8923076923077
episode 90 current score 73.0 avg score 98.9010989010989 best_score 131.8923076923077
episode 91 current score 213.0 avg score 100.1413043478261 best_score 131.8923076923077
episode 92 current score 135.0 avg score 100.51612903225806 best_score 131.8923076923077
episode 93 current score 126.0 avg score 100.7872340425532 best_score 131.8923076923077
episode 94 current score 174.0 avg score 101.5578947368421 best_score 131.8923076923077
episode 95 current score 178.0 avg score 102.35416666666667 best_score 131.8923076923077
episode 96 current score 156.0 avg score 102.90721649484536 best_score 131.8923076923077
episode 97 current score 184.0 avg score 103.73469387755102 best_score 131.8923076923077
episode 98 current score 236.0 avg score 105.07070707070707 best_score 131.8923076923077
episode 99 current score 242.0 avg score 106.44 best_score 131.8923076923077
episode 100 current score 198.0 avg score

episode 193 current score 10.0 avg score 246.1 best_score 250.29
episode 194 current score 12.0 avg score 244.48 best_score 250.29
episode 195 current score 33.0 avg score 243.03 best_score 250.29
episode 196 current score 35.0 avg score 241.82 best_score 250.29
episode 197 current score 105.0 avg score 241.03 best_score 250.29
episode 198 current score 353.0 avg score 242.2 best_score 250.29
episode 199 current score 370.0 avg score 243.48 best_score 250.29
episode 200 current score 311.0 avg score 244.61 best_score 250.29
episode 201 current score 500.0 avg score 247.96 best_score 250.29
episode 202 current score 500.0 avg score 247.96 best_score 250.29
saving model file
episode 203 current score 500.0 avg score 251.55 best_score 251.55
saving model file
episode 204 current score 500.0 avg score 254.9 best_score 254.9
episode 205 current score 101.0 avg score 254.88 best_score 254.9
saving model file
episode 206 current score 428.0 avg score 257.71 best_score 257.71
saving model file

KeyboardInterrupt: 

In [ ]:
for ep in range(episodes):
    state = env.reset()
    done = False
    score = 0
    while not done:
        action,prob,val = agent.choose_action(state)
        next_state,reward,done,_ = env.step(action)   
        score+=reward
        n_step+=1 
        state = next_state  
        env.render()
    print(f"ep score = {score}")


ep score = 425.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 416.0
ep score = 500.0
ep score = 462.0
ep score = 500.0
ep score = 500.0
ep score = 393.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 500.0
ep score = 500